## Step 1: Import Libraries

In [1]:
%run ../make_clean_names.py

In [2]:
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import logging

import polars as pl
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_session():
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    session.mount('https://', HTTPAdapter(max_retries=retries, pool_maxsize=10))
    return session

# Get API key from environment variables
FMP_API_KEY = os.getenv('FMP_API_KEY')
if not FMP_API_KEY:
    raise ValueError("FMP_API_KEY not found in environment variables")

## Step 2: Import Symbols

In [3]:
def load_symbols(file_path: str) -> List[str]:
    """Load symbols from a text file"""
    try:
        with open(file_path, 'r') as f:
            symbols = [line.strip() for line in f if line.strip()]
        print(f"Loaded {len(symbols)} symbols from {file_path}")
        return symbols
    except Exception as e:
        print(f"Error loading symbols: {str(e)}")
        return []

symbols_file = '../tickers.txt'
symbols = load_symbols(symbols_file)

if symbols:
    print("Symbols:", symbols)
else:
    print("No symbols loaded.")

Loaded 61 symbols from ../tickers.txt
Symbols: ['AAPL', 'MSFT', 'GOOGL', 'GOOG', 'META', 'NVDA', 'AVGO', 'ORCL', 'CRM', 'ACN', 'ADBE', 'CSCO', 'INTC', 'NFLX', 'DIS', 'CMCSA', 'VZ', 'T', 'AMZN', 'TSLA', 'HD', 'MCD', 'NKE', 'SBUX', 'TGT', 'LOW', 'WMT', 'PG', 'KO', 'PEP', 'COST', 'BRK-B', 'JPM', 'BAC', 'WFC', 'GS', 'MS', 'BLK', 'UNH', 'JNJ', 'PFE', 'ABBV', 'MRK', 'LLY', 'CAT', 'BA', 'HON', 'UPS', 'RTX', 'GE', 'XOM', 'CVX', 'COP', 'SLB', 'LIN', 'APD', 'ECL', 'PLD', 'AMT', 'CCI', 'OSW']


## Step 3: Extract Data from FMP into Polars

In [4]:
def fetch_company_profile(symbol: str, api_key: str, session: requests.Session) -> Dict:
    """Fetch company profile data from FMP API"""
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}"
    params = {"apikey": api_key}
    
    try:
        response = session.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        logger.error(f"Error fetching {symbol}: {str(e)}")
        return None

def fetch_all_profiles(symbols: List[str], api_key: str) -> List[Dict]:
    session = create_session()
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [
            executor.submit(fetch_company_profile, symbol, api_key, session)
            for symbol in symbols
        ]
        results = [
            f.result() for f in concurrent.futures.as_completed(futures)
        ]
    
    return [r[0] for r in results if r and isinstance(r, list)]

# Execute fetching
company_profiles = fetch_all_profiles(symbols, FMP_API_KEY)

# Convert to Polars DataFrame more efficiently
if company_profiles:
    df = pl.DataFrame(company_profiles)

ERROR:__main__:Error fetching AVGO: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/AVGO?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching GOOG: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/GOOG?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching NVDA: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/NVDA?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching META: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/META?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching ORCL: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/ORCL?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


...

ERROR:__main__:Error fetching TSLA: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/TSLA?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching CSCO: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/CSCO?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching DIS: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/DIS?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching T: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/T?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching AMZN: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/AMZN?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


## Step 4: Clean Column Names

In [5]:
df = make_clean_names(df)

## Step 5: Write Polars to Parquet

In [6]:
output_dir = "../../../data/finance"

# Write DataFrame to Parquet
df.write_parquet(f'{output_dir}/company_profile.parquet')

## Step 6: Read Parquet (Validate)

In [7]:
pl.scan_parquet(f'{output_dir}/company_profile.parquet').head().collect()

symbol,price,beta,vol_avg,mkt_cap,last_div,range,changes,company_name,currency,cik,isin,cusip,exchange,exchange_short_name,industry,website,description,ceo,sector,country,full_time_employees,phone,address,city,state,zip,dcf_diff,dcf,image,ipo_date,default_image,is_etf,is_actively_trading,is_adr,is_fund
str,f64,f64,i64,i64,f64,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,bool,bool,bool,bool,bool
"""NKE""",70.84,1.035,11612091,104782003378,1.6,"""70.32-107.43""",-0.27,"""NIKE, Inc.""","""USD""","""0000320187""","""US6541061031""","""654106103""","""New York Stock Exchange""","""NYSE""","""Apparel - Footwear & Accessori…","""https://investors.nike.com""","""NIKE, Inc., together with its …","""Mr. Elliott J. Hill""","""Consumer Cyclical""","""US""","""79400""","""503 671 6453""","""One Bowerman Drive""","""Beaverton""","""OR""","""97005-6453""",-15.071,85.910996,"""https://images.financialmodeli…","""1980-12-02""",false,false,true,false,false
"""HD""",409.38,1.011,3312144,406662944940,9.0,"""323.77-439.37""",0.05,"""The Home Depot, Inc.""","""USD""","""0000354950""","""US4370761029""","""437076102""","""New York Stock Exchange""","""NYSE""","""Home Improvement""","""https://www.homedepot.com""","""The Home Depot, Inc. operates …","""Mr. Edward P. Decker""","""Consumer Cyclical""","""US""","""465000""","""770 433 8211""","""2455 Paces Ferry Road""","""Atlanta""","""GA""","""30339""",87.72055,321.65945,"""https://images.financialmodeli…","""1981-09-22""",false,false,true,false,false
"""WMT""",91.94,0.516,16045842,738589876600,0.83,"""53.44-96.18""",0.64,"""Walmart Inc.""","""USD""","""0000104169""","""US9311421039""","""931142103""","""New York Stock Exchange""","""NYSE""","""Discount Stores""","""https://corporate.walmart.com""","""Walmart Inc. engages in the op…","""Mr. C. Douglas McMillon""","""Consumer Defensive""","""US""","""2100000""","""479 273 4000""","""702 South West 8th Street""","""Bentonville""","""AR""","""72716""",50.24918,41.690823,"""https://images.financialmodeli…","""1972-08-25""",false,false,true,false,false
"""LOW""",261.06,1.098,2381931,147407529000,4.5,"""209.55-287.01""",2.28,"""Lowe's Companies, Inc.""","""USD""","""0000060667""","""US5486611073""","""548661107""","""New York Stock Exchange""","""NYSE""","""Home Improvement""","""https://www.lowes.com""","""Lowe's Companies, Inc., togeth…","""Mr. Marvin R. Ellison""","""Consumer Cyclical""","""US""","""300000""","""704 758 1000""","""1000 Lowe's Boulevard""","""Mooresville""","""NC""","""28117""",123.44344,137.616557,"""https://images.financialmodeli…","""1980-03-17""",false,false,true,false,false
"""COST""",943.19,0.789,1912567,418681097810,4.5,"""675.96-1008.25""",23.44,"""Costco Wholesale Corporation""","""USD""","""0000909832""","""US22160K1051""","""22160K105""","""NASDAQ Global Select""","""NASDAQ""","""Discount Stores""","""https://www.costco.com""","""Costco Wholesale Corporation, …","""Mr. Ron M. Vachris""","""Consumer Defensive""","""US""","""333000""","""425 313 8100""","""999 Lake Drive""","""Issaquah""","""WA""","""98027""",496.76958,446.420425,"""https://images.financialmodeli…","""1986-07-09""",false,false,true,false,false
